In [2]:
import numpy as np
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
model_iMM904 = read_sbml_model('iMM904.xml')
phy = Metabolite(id='phy', name='phytoene', compartment='c') #Adding the metabolite
new_reaction1 = Reaction('phy_syn') #PHYtoene SYNtase
phy = Metabolite(id='phy', name='phytoene', compartment='c') #Adding the metabolite

new_reaction1.add_metabolites({model_iMM904.metabolites.ggdp_c: -2,
                              phy: 1, # our newly created metabolites
                               model_iMM904.metabolites.ppi_c: 2
                                })
model_iMM904.add_reactions([new_reaction1])
new_reaction2 = Reaction('neu_syn') #NEUrosporene SYNtase
#Adding the metabolite
neu = Metabolite(id='neu', name='Neurosporene', compartment='c') 
#AH2 = Metabolite(id='AH2', name='Reduced acceptor', compartment='c') 
#A = Metabolite(id='A', name='Acceptor', compartment='c') 
new_reaction2.add_metabolites({
    model_iMM904.metabolites.nadp_c: -3,
    phy: -1, # our newly created metabolites
    model_iMM904.metabolites.nadph_c: 3,
    neu: 1,
    })
model_iMM904.add_reactions([new_reaction2])
new_reaction3 = Reaction('lyc_syn') #LYCopene SYNtase
lyc = Metabolite(id='lyc', name='Lycopene', compartment='c') #Adding the metabolite
new_reaction3.add_metabolites({
    neu: -1, # our newly created metabolites
    model_iMM904.metabolites.nadph_c: -1, 
    model_iMM904.metabolites.o2_c: -1,
    lyc: 1,
    model_iMM904.metabolites.nadp_c: 1,
    model_iMM904.metabolites.h2o_c: 2,
    })
model_iMM904.add_reactions([new_reaction3])
new_reaction4 = Reaction('beta_syn') #BETA-carotene SYNtase
beta = Metabolite(id='beta', name='Beta-carotene', compartment='c') #Adding the metabolite
new_reaction4.add_metabolites({
    lyc: -1, # our newly created metabolites
    beta: 1,
    })
model_iMM904.add_reactions([new_reaction4])
new_reaction5 = Reaction('EX_beta') #BETA-carotene SYNtase
new_reaction5.add_metabolites({
    beta: -1 #Excretion of beta-carotene
    })
model_iMM904.add_reactions([new_reaction5])
with model_iMM904:
    model_iMM904.objective = model_iMM904.reactions.EX_beta 
    print("Maximum beta-carotene flow: ",model_iMM904.optimize().objective_value)

Maximum beta-carotene flow:  0.35143769968051247


In [3]:
carbon_source_exchanges=[]
for exchange in model_iMM904.exchanges:
    for metabolite in exchange.metabolites:
        if "C" in metabolite.elements:
            carbon_source_exchanges.append(exchange.id)
print(carbon_source_exchanges)

['EX_epistest_SC_e', 'EX_epist_e', 'EX_ergst_e', 'EX_ergstest_SC_e', 'EX_13BDglcn_e', 'EX_etha_e', 'EX_2hb_e', 'EX_etoh_e', 'EX_fecost_e', 'EX_2mbac_e', 'EX_2mbald_e', 'EX_2mbtoh_e', 'EX_2mppal_e', 'EX_2phetoh_e', 'EX_fecostest_SC_e', 'EX_fmn_e', 'EX_3c3hmp_e', 'EX_3mbald_e', 'EX_for_e', 'EX_fru_e', 'EX_3mop_e', 'EX_4abut_e', 'EX_fum_e', 'EX_g3pc_e', 'EX_4abz_e', 'EX_5aop_e', 'EX_g3pi_e', 'EX_gal_e', 'EX_8aonn_e', 'EX_Nbfortyr_e', 'EX_abt_e', 'EX_ac_e', 'EX_acald_e', 'EX_aces_e', 'EX_galur_e', 'EX_gam6p_e', 'EX_gcald_e', 'EX_glc__D_e', 'EX_gln__L_e', 'EX_glu__L_e', 'EX_glx_e', 'EX_ade_e', 'EX_adn_e', 'EX_akg_e', 'EX_ala__L_e', 'EX_gly_e', 'EX_glyc_e', 'EX_gsn_e', 'EX_gthox_e', 'EX_gthrd_e', 'EX_alltn_e', 'EX_alltt_e', 'EX_amet_e', 'EX_arab__D_e', 'EX_arab__L_e', 'EX_gua_e', 'EX_arg__L_e', 'EX_hdca_e', 'EX_hdcea_e', 'EX_asn__L_e', 'EX_asp__L_e', 'EX_btd_RR_e', 'EX_btn_e', 'EX_hexc_e', 'EX_his__L_e', 'EX_hxan_e', 'EX_iamac_e', 'EX_iamoh_e', 'EX_ibutac_e', 'EX_chol_e', 'EX_cit_e', 'EX_co2

In [26]:
aerobic_carbon_sources=[]
for i in range (0,len(carbon_source_exchanges)):
    with model_iMM904:
        medium=model_iMM904.medium
        medium['EX_glc__D_e']=0
        medium[carbon_source_exchanges[i]]=100
        model_iMM904.medium =medium
        solution=model_iMM904.slim_optimize(error_value=0.)
        if solution >0:
            aerobic_carbon_sources.extend([carbon_source_exchanges[i]])
            print(carbon_source_exchanges[i])
            print(solution)
print(aerobic_carbon_sources)

EX_13BDglcn_e
2.280376608317388
EX_2hb_e
0.04278624497058172
EX_etoh_e
0.029162766956666083
EX_3c3hmp_e
2.45044350629841e-16
EX_for_e
0.015680467905162265
EX_fru_e
2.280376608318034
EX_4abut_e
0.04596972510305107
EX_fum_e
0.3353673362275545
EX_gal_e
2.280376608317796
EX_ac_e
0.025575573485879768
EX_acald_e
0.029162766956663772
EX_gam6p_e
3.379353287025532
EX_glc__D_e
2.2803766083180257
EX_gln__L_e
0.06324141092047614
EX_glu__L_e
0.0626503596766645
EX_glx_e
0.030856104605897387
EX_adn_e
3.0602041876131865
EX_akg_e
0.07702180307527515
EX_ala__L_e
0.0439218096806771
EX_gly_e
0.04428403291587839
EX_glyc_e
0.08606133583751305
EX_gsn_e
3.039423765163895
EX_alltn_e
0.03085610460589758
EX_alltt_e
0.030856104605897574
EX_amet_e
3.0763187173161084
EX_arg__L_e
0.046205626151634815
EX_asn__L_e
0.3432692367300105
EX_asp__L_e
0.3420340051681373
EX_cit_e
0.0792462309832161
EX_cytd_e
0.06808536848942173
EX_ile__L_e
5.900530771791386e-20
EX_ins_e
3.0528816446780866
EX_lac__D_e
0.039203956604434737
EX_l

In [27]:
aerobic_carbon_sources=[]
for i in range (0,len(carbon_source_exchanges)):
    with model_iMM904:
        medium=model_iMM904.medium
        medium['EX_glc__D_e']=0
        medium[carbon_source_exchanges[i]]=100
        model_iMM904.medium =medium
        solution=model_iMM904.slim_optimize(error_value=0.)
        if solution >2:
            aerobic_carbon_sources.extend([carbon_source_exchanges[i]])
            print(carbon_source_exchanges[i])
            print(solution)
            model_iMM904.objective = model_iMM904.reactions.EX_beta 
            print("Maximum beta-carotene flow: ",model_iMM904.optimize().objective_value)
print(aerobic_carbon_sources)

EX_13BDglcn_e
2.2803766083180457
Maximum beta-carotene flow:  1.9999999999999991
EX_fru_e
2.2803766083180266
Maximum beta-carotene flow:  2.0000000000000004
EX_gal_e
2.2803766083180386
Maximum beta-carotene flow:  1.9999999999999998
EX_gam6p_e
3.379353287025514
Maximum beta-carotene flow:  1.9999999999998987
EX_glc__D_e
2.280376608318025
Maximum beta-carotene flow:  1.9999999999999998
EX_adn_e
3.0602041876131714
Maximum beta-carotene flow:  2.0000000000000058
EX_gsn_e
3.0394237651638996
Maximum beta-carotene flow:  1.9999999999999993
EX_amet_e
3.0763187173160755
Maximum beta-carotene flow:  2.0000000000000053
EX_ins_e
3.052881644678164
Maximum beta-carotene flow:  1.9999999999999996
EX_malt_e
4.478329965732987
Maximum beta-carotene flow:  2.0000000000000164
EX_man_e
2.280376608318027
Maximum beta-carotene flow:  2.0000000000000004
EX_melib_e
4.478329965732991
Maximum beta-carotene flow:  2.000000000000001
EX_sucr_e
4.47832996573299
Maximum beta-carotene flow:  2.0
EX_pap_e
3.0648087057

In [9]:
len(carbon_source_exchanges)

154

In [18]:
len(aerobic_carbon_sources)

16

Udvid til at sortere/tjekke beta-caroten flowet. Also tjek for anaerobe